Prvi dio: Treniranje neuronske mreže

In [1]:
import torch as T
import torch.nn as nn

Koristimo Covertype skup podataka tokom treniranja neuronske mreže. Data je arhitektura same mreže i funkcija propagacije unaprijed.

In [2]:
class Covertype(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Linear(54,216)
        self.act1=nn.ReLU()
        #self.drop1=nn.Dropout(p=0.1)
        self.layer2 = nn.Linear(216, 864)
        self.act2 = nn.ReLU()
        #self.drop2 = nn.Dropout(p=0.1)
        self.layer3 = nn.Linear(864, 378)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(378, 7)
        self.output_act = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.act1(self.layer1(x))
       # x = self.drop1(x)
        x = self.act2(self.layer2(x))
        #x = self.drop2(x)
        x = self.act3(self.layer3(x))
        #x = self.output(x)
        x = nn.functional.softmax(self.output(x),dim=1)
        return x

In [3]:
from sklearn.datasets import fetch_covtype

covertype=fetch_covtype()
X = covertype.data
y = covertype.target 

Definisaćemo trening i testni skup podataka.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [5]:
#type(X_train)

In [6]:
from torch.utils.data import TensorDataset, DataLoader

X_train_t = T.tensor(X_train).float()
X_test_t = T.tensor(X_test).float()
y_train_t = T.tensor(y_train).float().reshape(-1, 1)
y_test_t = T.tensor(y_test).float().reshape(-1, 1)

In [7]:
#Provjera
#print(X_train_t.shape)
#print(y_train_t.shape)

In [8]:
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [10]:
device = T.device("cuda:0") if T.cuda.is_available() else T.device("cpu") 
print(device)

cpu


In [11]:
net = Covertype().to(device)

In [12]:
#Koristtimo Adam optimizaciju za parametre 
optimizer = T.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
#  Definisemo adekvatnu funkciju greske za problem
loss_fn = nn.CrossEntropyLoss()

Testni model, importujem već gotove implementacije za odziv i preciznost i poredimo dobijene vrijednosti prije i nakon treniranja mreže.

In [13]:
from sklearn.metrics import recall_score,precision_score
def test_model(net, loss_fn, test_loader):
    total_loss = 0
    net.eval()
    true_labels = []
    predicted_labels = []

    with T.no_grad():
        for x, y in test_loader:
            x = x.float().to(device)
            y = y.long().squeeze().to(device)
            preds = net(x)
            loss = loss_fn(preds, y-1)
            total_loss += loss.item()

            a,pred_labels = T.max(preds, dim=1)
            true_labels.extend(pred_labels.numpy())
            predicted_labels.extend((y-1).numpy())

    avg_loss = total_loss / len(test_loader)
    #Zero division da izbjegnemo dijeljenje sa 0 i nedefinisanost
    recall = recall_score(true_labels, predicted_labels, average='weighted', zero_division=1)
    precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=1)

    return avg_loss, recall, precision

In [14]:
loss, recall, precision = test_model(net, loss_fn, test_loader)
#print("Loss:", loss)
print("Recall:", recall)
print("Precision:", precision)

Recall: 0.21281722502861372
Precision: 0.24730318544950441


In [15]:
EPOCHS=3

Treniranje neuronske mreže vršimo u svakoj od epoha.


In [16]:
net.train()
for i in range(EPOCHS):
    for x,y in train_loader:
        x=x.float().to(device)
        y=y.long().squeeze().to(device)
        preds=net(x)
        loss = loss_fn(preds,y-1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [17]:
loss, recall, precision = test_model(net, loss_fn, test_loader)
#print("Loss:", loss)
print("Recall:", recall)
print("Precision:", precision)

Recall: 0.48621808387046805
Precision: 1.0


Drugi dio:
Neophodno je da definišemo validacioni skup podataka kako bismo implementirali early stopping u toku treninga.



In [18]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_covtype

covertype=fetch_covtype()
X = covertype.data
y = covertype.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val= train_test_split(X_test,y_test,test_size=0.3, random_state=42)

In [19]:
X_train_t = T.tensor(X_train).float()
X_val_t = T.tensor(X_val).float()
y_train_t = T.tensor(y_train).long().reshape(-1)
y_val_t = T.tensor(y_val).long().reshape(-1)

train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

In [20]:
def early_stoping(net,train_loader,Val_loader,loss_fn,optimizer):
    num_epochs=10
    # Inicijalizacija brojaca  za early stopping
    early_stopping_counter = 0
    best_val_loss = float('inf')

    # Treniranje modela sa early stopping-pm
    for epoch in range(num_epochs):
        #Treniranje modela razlicitim skupom od pocetnog
        net.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            preds = net(x)
            loss = loss_fn(preds, y-1)
            loss.backward()
            optimizer.step()
        
        #Evaluacija modela nad validacionim skupom
        net.eval()
        val_loss = 0
        for x_val, y_val in val_loader:
                x_val = x_val.float().to(device)
                y_val = y_val.long().squeeze().to(device)
                val_preds = net(x_val)
                val_loss += loss.item()#ovo je ukupna greska tokom jedne epohe

        avg_val_loss = val_loss / len(val_loader)
        
        #Provjera da li je greska nad validacionim skupom pocela da raste, tj da li smo dostigli trecu uzastopnu epohu u kojoj greska raste
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if (early_stopping_counter == 3):
                print("Early stopping triggered. Training stopped.")
                break

        print(f"Epoch: {epoch+1}, Average Val Loss: {avg_val_loss}")
early_stoping(net,train_loader,val_loader,loss_fn,optimizer)


Epoch: 1, Average Val Loss: 1.7299383878707886
Epoch: 2, Average Val Loss: 1.7460672855377197
Epoch: 3, Average Val Loss: 1.6009060144424438
Epoch: 4, Average Val Loss: 1.6654220819473267
Epoch: 5, Average Val Loss: 1.617035150527954
Early stopping triggered. Training stopped.


Ostalo je da pretražimo sam prostor hiperparametara varirajućim vrijednostima. Definisala sam novu klasu da ne bi došlo do promjene na prethodnim vrijednostima.


In [21]:
class Covertype(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, hidden_size * 4)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(hidden_size * 4, hidden_size * 2)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(hidden_size * 2, output_size)
        self.output_act = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.output_act(self.output(x))
        return x


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_covtype

covertype=fetch_covtype()
X = covertype.data
y = covertype.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val= train_test_split(X_test,y_test,test_size=0.3, random_state=42)

In [23]:
X_train_t = T.tensor(X_train).float()
X_val_t = T.tensor(X_val).float()
y_train_t = T.tensor(y_train).long().reshape(-1)
y_val_t = T.tensor(y_val).long().reshape(-1)

train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


In [24]:
factors = [0.001, 0.01]  #Pocetne vrijednosti faktora ucenja
hidden_sizes = [128, 216]  #Broj neurona u skrivenim slojevima
batch_sizes = [32, 64]  #Velicina batch-a
best_precision = 0.0
best_recall = 0.0
best_model = None
best_parameters = None

for factor in factors:
    for hidden_size in hidden_sizes:
        for batch_size in batch_sizes:
            train_dataset = TensorDataset(X_train_t, y_train_t)
            val_dataset = TensorDataset(X_val_t, y_val_t)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True) 
            model = Covertype(input_size=54, hidden_size=216, output_size=7)
            loss_fn = nn.CrossEntropyLoss()
            optimizer = T.optim.Adam(model.parameters(), lr=factor)

            early_stoping(model, train_loader, val_loader, loss_fn, optimizer)

            a,precision, recall = test_model(model, loss_fn, val_loader)

            if precision > best_precision:
                best_precision = precision
                best_recall = recall
                best_model = model
                best_parameters = (factor, hidden_size, batch_size)

print("\nNajbolji slučajevi su pri vrijednostima:")
print("Faktor ucenja: {}".format(best_parameters[0]))
print("Broj neurona u skrivenom sloju: {}".format(best_parameters[1]))
print("Velicina batch-a: {}".format(best_parameters[2]))


Epoch: 1, Average Val Loss: 1.8654223680496216
Epoch: 2, Average Val Loss: 1.898755669593811
Epoch: 3, Average Val Loss: 1.8320889472961426
Epoch: 4, Average Val Loss: 1.6654222011566162
Epoch: 5, Average Val Loss: 1.898755669593811
Epoch: 6, Average Val Loss: 1.6987555027008057
Early stopping triggered. Training stopped.
Epoch: 1, Average Val Loss: 2.1492931842803955
Epoch: 2, Average Val Loss: 2.133164167404175
Epoch: 3, Average Val Loss: 2.133164167404175
Epoch: 4, Average Val Loss: 2.1009061336517334
Epoch: 5, Average Val Loss: 2.068648099899292
Epoch: 6, Average Val Loss: 2.133164167404175
Epoch: 7, Average Val Loss: 2.1009061336517334
Early stopping triggered. Training stopped.
Epoch: 1, Average Val Loss: 2.165422201156616
Epoch: 2, Average Val Loss: 2.165422201156616
Epoch: 3, Average Val Loss: 2.165422201156616
Epoch: 4, Average Val Loss: 2.1320888996124268
Epoch: 5, Average Val Loss: 2.1320888996124268
Epoch: 6, Average Val Loss: 2.1320888996124268
Epoch: 7, Average Val Loss: 